In [1]:
import os
from psycopg2 import connect  #postgre 연결용

import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point, MultiPolygon, Polygon, MultiLineString,MultiPoint
from geoalchemy2 import Geometry, WKTElement, Geography
from sqlalchemy import create_engine, types, engine
import shapely
from shapely import wkt
from shapely.wkt import loads

from module_sql_table import *
from connection_info import *

import datetime
from tqdm import tqdm
import time

#python의 64형식 정수, 실수 충돌방지
import numpy 
from psycopg2.extensions import register_adapter, AsIs
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)

#polygon 출동방지
from geoalchemy2.shape import from_shape
from shapely.geometry import Point, Polygon
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
from connection_info import con_engine

/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_3647/3625640409.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### TEST

In [2]:
# # 정상작동 확인
# cursor2.execute('select RELNAME AS TABLE_NAME\
#                FROM PG_STAT_USER_TABLES;')
# cursor2.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'  
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor2.fetchall()
# print(li_tab_col,'\n', len(li_tab_col))

In [3]:
# #테이블명 수집
# li_table = []
# for i in range(len(li_tab_col)):
#     li_table.append(li_tab_col[i][0]) #-> 각튜플의 0번째 자리는 테이블명
# li_table = list(set(li_table)) # set을 통해 튜플로 중복값을 제거

In [4]:
# li_whole = []
# for i in li_table:
#     geondba = postgre_con('geondb','10.0.3.13',5432,'geondba','@geondba')
#     cursor = geondba.cursor()
#     cursor.execute(f'''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                     FROM INFORMATION_SCHEMA.COLUMNS 
#                     WHERE TABLE_CATALOG = 'geondb'
#                     and table_schema = 'gfdata'
#                     and table_name = '{i}'
#                     ORDER BY 테이블이름, ORDINAL_POSITION;''')
#     li_tab_col = cursor.fetchall()
#     li = []
#     li.append(i) #각 리스트 맨 처음에 테이블명 삽입
#     for j in range(len(li_tab_col)):
#         li.append(li_tab_col[j][1]) #컬럼명 수집
#     li_whole.append(li)


In [5]:
# #파티셔닝 테이블 중 하나의 컬럼명을 통해 최신파일과의 컬럼 비교
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'lsmd_cont_ldreg'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# li

## 여기부터 테이블 생성 및 데이터 insert

## layer_group_1 법정동경계

In [6]:
# #layer_group_1_juso #tl_scco_ctpvn_new
# # 행정경계_법정동_시도
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_1_juso/TL_SCCO_CTPRVN' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_scco_ctprvn'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_scco_ctprvn_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186', geometry='geometry').reset_index()
#     # gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_scco_ctprvn_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [7]:
# cursor_gf.execute('''drop TABLE gfdata.tl_scco_ctprvn_new
#                 ''')
# gf.commit()

In [8]:
# #layer_group_1_juso #tl_scco_emd_new
# # 행정경계_법정동_읍면동
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_1_juso/TL_SCCO_EMD' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_scco_emd'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_scco_emd_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186', geometry='geometry').reset_index()
#     # gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_scco_emd_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         # print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [9]:
# cursor_gf.execute('''drop TABLE gfdata.tl_scco_emd_new
#                 ''')
# gf.commit()

In [10]:
# #layer_group_1_juso #tl_scco_li_new
# # 행정경계_법정동_리
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_1_juso/TL_SCCO_LI' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_scco_li'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_scco_li_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186', geometry='geometry').reset_index()
#     # gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_scco_li_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         # print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [11]:
# #layer_group1_juso #tl_scco_sig_new
# # 행정경계_법정동_시군구
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_1_juso/TL_SCCO_SIG' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_scco_sig'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_scco_sig_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186', geometry='geometry').reset_index()
#     # gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_scco_sig_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         # print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

## layer_group_1 행정동 경계 : 테이블 생성 및 데이터 입력 코드 작성 필요

In [12]:
# #layer_group_1_juso #z_sop_bnd_sido_pg_new
# # 행정경계_법정동_시도
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_1_juso/Z_SOP_BND_SIDO_PG' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'z_sop_bnd_sido_pg'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['z_sop_bnd_sido_pg_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5186', geometry='geometry').reset_index()
#     # gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.z_sop_bnd_sido_pg_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

## layer_group_2 연속지적도

In [13]:
li_table_names = [
                'lsmd_cont_ldreg_11_new', #테이블명 리스트
                'lsmd_cont_ldreg_26_new',
                'lsmd_cont_ldreg_27_new',
                'lsmd_cont_ldreg_28_new',
                'lsmd_cont_ldreg_29_new',
                'lsmd_cont_ldreg_30_new',
                'lsmd_cont_ldreg_31_new',
                'lsmd_cont_ldreg_36_new',
                'lsmd_cont_ldreg_41_new',
                'lsmd_cont_ldreg_43_new',
                'lsmd_cont_ldreg_44_new',
                'lsmd_cont_ldreg_45_new',
                'lsmd_cont_ldreg_46_new',
                'lsmd_cont_ldreg_47_new',
                'lsmd_cont_ldreg_48_new',
                'lsmd_cont_ldreg_50_new',
                'lsmd_cont_ldreg_51_new']

In [15]:
# # 4월 업데이트 이슈: 
# #---------------------------------------------------------------
# # 마스터 및 파티션 테이블 생성 후 단계: 자료가공 후 insert 시, 인덱스(pnu 조건과 데이터 pnu 가 안맞는듯?)
# # -> 테이블 순거와 데이터 순서가 맞지 않음, 여기는 int 필드에 nan이 있는 게 아니라 그냥 쓸 수 없는 pnu가 섞여있음 
# # -> dic_seq_earth의 내용 확인, key-value 쌍 맞는지->여기가 문제였음!!, 본 스크립트 실행 전, 폴더명 바꿔야할듯 
# #---------------------------------------------------------------

# # 5월 업데이트 이슈: pnu가 +1 되어있는 레이어들있음 45부터 쭉 밀리고 있음(insert 안됨) -> 해결
# # li_table_names에서 45이전부터는 주석처리 후 다시 코드 돌리기 -> 해결
# # 그냥 전북특자도는 로컬 행정구역 코드 넣을 때 45로 넣어주자


# # 연속지적도 업로드
# from tqdm import tqdm
# import time
# from shapely.geometry import MultiPolygon
# # 통 테이블 가져오기

# path = '../layer_group_2/' #나머지 path(시도별 폴더명은 각각 넣어주기)

# # ----------------------------------------------------------------------------------------------
# #파티션테이블 생성 반복문
# for i in li_table_names: 
#     if i.split('_')[3] != '45':
#         j = i.split('_')[3]
#     else:
#         j = '52' #-> 전북특자도 pnu 조건 52 시작으로 변경, partition table명은 45로
    
#     dic[i] = f'''CREATE TABLE gfdata.{i} PARTITION OF gfdata.lsmd_cont_ldreg_new
#     FOR VALUES FROM ('{j}00000000000000000') TO ('{int(j)+1}00000000000000000');
# 	CREATE INDEX lsmd_cont_ldreg_{i.split('_')[3]}_new_geom_idx_{time_1[-2:]} ON gfdata.lsmd_cont_ldreg_{i.split('_')[3]}_new USING gist (geom);
# 	CREATE INDEX lsmd_cont_ldreg_{i.split('_')[3]}_new_pnu_idx_{time_1[-2:]} ON gfdata.lsmd_cont_ldreg_{i.split('_')[3]}_new USING btree (pnu);'''
# # ----------------------------------------------------------------------------------------------


# # 하위폴더명 가져오기
# li_file_names = os.listdir('../layer_group_2')
# try:
#     li_file_names.remove('.DS_Store')
# except:
#     pass 
# dic_sq_earth = {}

# li_file_names = sorted(li_file_names)

# #check_point
# for i, j in zip(sorted(li_table_names), sorted(li_file_names)):
#     dic_sq_earth[i] = j 

# # table = lsmd_cont_ldreg_new (파티션 상위테이블)
# # 연속지적도 좌표계: 5186, prj 파일은 없음
# # encoding = cp949?

# # # ----------------------------------------------------------------------------------------------
# sql = dic['lsmd_cont_ldreg_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #하위테이블 생성
# for i in li_table_names:
#     sql = dic[i].replace('\n','').replace('\t','')    
#     cursor_gf.execute(sql)
#     gf.commit()   

# #위 과정 마무리 시 인덱스 생성해야함
# sql = f'''CREATE INDEX lsmd_cont_ldreg_new_geom_idx_{time_1[-2:]} ON ONLY gfdata.lsmd_cont_ldreg_new USING gist (geom);
# CREATE INDEX lsmd_cont_ldreg_new_pnu_idx_{time_1[-2:]} ON ONLY gfdata.lsmd_cont_ldreg_new USING btree (pnu);'''

# cursor_gf.execute(sql)
# gf.commit()

# # ----------------------------------------------------------------------------------------------

# #컬럼명 가져오기
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'lsmd_cont_ldreg_all'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]
# li_upper = [i.upper() for i in li]
# li_upper[-1] = 'geometry'


# #data_read 및 insert
# for i in tqdm(sorted(li_table_names)):
#     print(i)
#     nm = i
#     file = gpd.read_file(f'../layer_group_2/{dic_sq_earth[i]}', encoding = 'cp949')
#     file['len'] = file['PNU'].apply(lambda x: len(x))
#     #PNU의 자리수와 시작 2자를 통해 유효하지 않은 데이터를 필터링하는 부분
#     file_1 = file.loc[(file['len']==19)&(file.PNU.str[:2] == file.COL_ADM_SE.str[:2])]
#     print('정상데이터비율 : ',len(file_1)/len(file)*100)
#     file_2 = file_1[li_upper]
#     gdf = gpd.GeoDataFrame(file_2, crs = 'EPSG:5186', geometry='geometry').reset_index(drop = True)
#     data = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(data)):
#         polygon = data[i][-1]
#         try:
#             data[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             data[i][-1] = str(from_shape(polygon, srid=5186))
#     print('데이터길이:',len(data))
#     for j in (range(0,len(data),250000)): 
#     #insert
#         if j+250000 <= len(data): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[j:j+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[j:]])
#         sql = "INSERT INTO gfdata.{nm} ({col}) VALUES {d};".format(nm = nm,col = col,d=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#         print(j/len(data)*100)
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))    

  0%|          | 0/17 [00:00<?, ?it/s]

lsmd_cont_ldreg_11_new
정상데이터비율 :  100.0
데이터길이: 903843
0.0
27.659670982681728
55.319341965363456


  6%|▌         | 1/17 [02:43<43:31, 163.21s/it]

82.97901294804518
Taken Time: 14.73228 secs
lsmd_cont_ldreg_26_new
정상데이터비율 :  99.99985994770448
데이터길이: 714018
0.0
35.01312291846984


 12%|█▏        | 2/17 [04:56<36:24, 145.63s/it]

...

98.55323845941578
Taken Time: 7.67938 secs
lsmd_cont_ldreg_50_new
정상데이터비율 :  99.99988636066625
데이터길이: 879976
0.0
28.409865723610643
56.81973144722129


 94%|█████████▍| 16/17 [1:54:53<10:45, 645.12s/it]

85.22959717083192
Taken Time: 12.63878 secs
lsmd_cont_ldreg_51_new
정상데이터비율 :  99.99985343889153
데이터길이: 2729233
0.0
9.160082704554723
18.320165409109446
27.480248113664167
36.64033081821889
45.80041352277362
54.960496227328335
64.12057893188306
73.28066163643778
82.44074434099251


100%|██████████| 17/17 [2:03:11<00:00, 434.80s/it]

91.60082704554723
Taken Time: 21.34052 secs


In [14]:
# 기존 table 뒤에 old 붙여주기
for i in li_table_names:
    newless = '_'.join(i.split('_')[:-1])
    cursor_gf.execute(f'''alter TABLE gfdata.{newless} rename to {newless}_202404;''')
    gf.commit()
# 적재한 table, db 상 이름으로 바꿔주기
for i in li_table_names:
    newless = '_'.join(i.split('_')[:-1])
    cursor_gf.execute(f'''alter TABLE gfdata.{i} rename to {newless};''')
    gf.commit()
# 마스터 테이블명 변경
cursor_gf.execute('''alter TABLE gfdata.lsmd_cont_ldreg_all rename to lsmd_cont_ldreg_all_202404;''')
cursor_gf.execute('''alter TABLE gfdata.lsmd_cont_ldreg_new rename to lsmd_cont_ldreg_all;''')
gf.commit()

In [14]:
cursor_gf.execute('''drop TABLE gfdata.lsmd_cont_ldreg_new;
                ''')
gf.commit()

## layer_group_3 철도

In [17]:
# #data 만들기 table = tl_spsb_statn_new
# #교통_지하철: 좌표 = 5179
# #encoding = cp949
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_sttn/TL_SPSB_STATN'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# d = []
# data = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌 
#     data.extend(d)
# for i in range(len(data)):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5179))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5179))    

# # ------------------------------------------------------------------------------ 
# #테이블 생성
# sql = dic['tl_spsb_statn_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()
# #ls         
# import time
# from shapely.geometry import Point, MultiPolygon
# #insert
# args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data])
# # query = "gfdata.lsmd_cont_ldreg_s (gid ,pnu, jibun, bchk, sgg_oid,col_adm_sect_cd,geom) values(%s,%s,%s,%s,%s,%s,%s)"
# # print(cursor_gf.mogrify(query,data[1]))
# # cursor_gf.executemany(query,data)
# sql = "INSERT INTO gfdata.tl_spsb_statn_new (gid, sig_cd, sub_sta_sn, kor_sub_nm, opert_de, geom) VALUES {i};".format(i=args_str)

# start_time = time.time()
# try:
#     cursor_gf.execute(sql)
# except Exception as e:
#     print("Insert Error: ", e)

# gf.commit()
# print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [18]:
# cursor_gf.execute('''drop TABLE gfdata.tl_spsb_statn_new
#                 ''')
# gf.commit()

In [19]:
# #data 만들기 table = tl_spsb_entrc_new
# #교통_지하철: 좌표 = 5179
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# sql = dic['tl_spsb_entrc_new']
# path = '../layer_group_3_sttn/TL_SPSB_ENTRC'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# data = []
# d = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]))
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     gdf.to_crs(epsg = 5186, inplace = True)
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][-1] = str(from_shape(d[i][-1], srid = 5186))
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌
#     data.extend(d)
# # ------------------------------------------------------------------------------ 
# #테이블 생성
# sql = dic['tl_spsb_entrc_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()
# #ls
# import time
# from shapely.geometry import Point, MultiPolygon
# #insert
# args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data])
# sql = "INSERT INTO gfdata.tl_spsb_entrc_new (gid, sig_cd, sub_ent_sn, entrc_no, opert_de, geom) VALUES {i};".format(i=args_str)

# start_time = time.time()
# try:
#     cursor_gf.execute(sql)
# except Exception as e:
#     print("Insert Error: ", e)

# gf.commit()
# print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))


In [20]:
# cursor_gf.execute('''drop TABLE gfdata.tl_spsb_entrc_new
#                 ''')
# gf.commit()

In [21]:
# #data 만들기 table = tl_spsb_rlway_new
# #교통_지하철: 좌표 = 5179
# #encoding = cp949
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_sttn/TL_SPSB_RLWAY'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# d = []
# data = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     gdf.to_crs(epsg = 5186, inplace = True)
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌 
#     data.extend(d)
# for i in range(len(data)):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiLineString([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186)) 
# # ------------------------------------------------------------------------------ 
# #테이블 생성
# sql = dic['tl_spsb_rlway_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()
# #ls
# # #컬럼명 가져오기
# li = []
# cursor_gf.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_spsb_rlway'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor_gf.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]    

# # #insert
# args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data])
# sql = "INSERT INTO gfdata.tl_spsb_rlway_new ({col}) VALUES {i};".format(col = col,i=args_str)
 
# start_time = time.time()
# try:
#     cursor_gf.execute(sql)
# except Exception as e:
#     print("Insert Error: ", e)

# gf.commit()
# print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [22]:
# cursor_gf.execute('''drop TABLE gfdata.tl_spsb_rlway_new
#                 ''')
# gf.commit()

In [23]:
# #data 만들기 table = tl_sprl_statn_new
# #교통_지하철: 좌표 = 5179
# #encoding = cp949
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_sttn/TL_SPRL_STATN'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# d = []
# data = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     gdf.to_crs(epsg = 5186, inplace = True)
#     n+=len(d)
#     print(n)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#         d[i][0] = d[i][0] + n  # Adjusting the index
#         # Convert the last element (geometry) to Polygon if it's a MultiPolygon
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186))
  
#     data.extend(d)
# # ------------------------------------------------------------------------------ 
# #테이블 생성
# sql = dic['tl_sprl_statn_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()
# #ls
# import time
# from shapely.geometry import Point, MultiPolygon
# #insert
# args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data])

# sql = "INSERT INTO gfdata.tl_sprl_statn_new (gid, sig_cd, rlr_sta_sn, kor_sta_nm, opert_de, geom) VALUES {i};".format(i=args_str)

# start_time = time.time()
# try:
#     cursor_gf.execute(sql)
# except Exception as e:
#     print("Insert Error: ", e)

# gf.commit()
# print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [24]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sprl_statn_new
#                 ''')
# gf.commit()

In [25]:
# #data 만들기 table = tl_sprl_rlway_new
# #교통_지하철: 좌표 = 5179
# #encoding = cp949
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_sttn/TL_SPRL_RLWAY'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# d = []
# data = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     gdf.to_crs(epsg = 5186, inplace = True)
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌 
#     data.extend(d)
# for i in range(len(data)):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiLineString([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186))    

# # ------------------------------------------------------------------------------ 
# #테이블 생성
# sql = dic['tl_sprl_rlway_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #insert
# args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data])

# #컬럼명 가져오기
# li = []
# cursor_gf.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_sprl_rlway'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor_gf.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# sql = "INSERT INTO gfdata.tl_sprl_rlway_new ({col}) VALUES {i};".format(col = col,i=args_str)

# start_time = time.time()
# try:
#     cursor_gf.execute(sql)
# except Exception as e:
#     print("Insert Error: ", e)

# gf.commit()
# print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [26]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sprl_rlway_new
#                 ''')
# gf.commit()

## layer_group_3 도로

In [27]:
# # 도로교통-도로(기초구간)
# # # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_road/TL_SPRD_INTRVL'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# d = []
# data = []
# n = 0 #index를 먹이기 위함
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs ='EPSG:5179', geometry='geometry').reset_index()
#     gdf.to_crs(epsg = 5186, inplace = True)
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌 
#     data.extend(d)
# for i in range(len(data)):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiLineString([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186))    

# #테이블 생성
# sql = dic['tl_sprd_intrvl_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor_gf.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_sprd_intrvl'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor_gf.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# for i in tqdm(range(0,len(data),250000)): 
# #insert
#     if i+250000 <= len(data): 
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:i+250000]])
#     else:
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:]])
#     sql = "INSERT INTO gfdata.tl_sprd_intrvl_new ({col}) VALUES {i};".format(col = col,i=args_str)
#     start_time = time.time()
#     try:
#         cursor_gf.execute(sql)
#     except Exception as e:
#         print("Insert Error: ", e)
#     gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [28]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sprd_intrvl_new;''')
# gf.commit()

In [29]:
# # 도로교통-도로(도로구간)
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_road/TL_SPRD_MANAGE'
# li_folder = os.listdir(path) #지역별 지하철역 위치 폴더명 반환
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# data = []
# n = 0 #index를 먹이기 위함
# d = []
# for i in li_folder:
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5179', geometry='geometry').reset_index()
#     gdf = gdf.to_crs('EPSG:5186')
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     print(f'{i} 데이터 가공')
#     for i in tqdm(range(len(d))):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌
#     data.extend(d)
# print(' "DATA" 가공')   
# for i in tqdm(range(len(data))):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiLineString([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186)) 

# #테이블 생성
# sql = dic['tl_sprd_manage_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_sprd_manage'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]#---------------------------------------------------------------------------------------------------------

# #insert
# for i in tqdm(range(0,len(data),250000)):
#     if i+250000 <= len(data): 
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:i+250000]])
#     else:
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:]])
#     sql = "INSERT INTO gfdata.tl_sprd_manage_new ({col}) VALUES {i};".format(col = col,i=args_str)
#     start_time = time.time()
#     try:
#         cursor_gf.execute(sql)
#     except Exception as e:
#         print("Insert Error: ", e)
#     gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))



In [30]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sprd_manage_new;''')
# gf.commit()

In [31]:
# # 도로교통-도로(실폭도로)
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_3_road/TL_SPRD_RW'
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# data = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5179', geometry='geometry').reset_index()
#     gdf = gdf.to_crs('EPSG:5186')
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌
#     data.extend(d)
# print('DATA 가공')
# for i in tqdm(range(len(data))):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186)) 

# #테이블 생성
# sql = dic['tl_sprd_rw_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #insert

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_sprd_rw'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]
# #---------------------------------------------------------------------------------------------------------

# #insert
# for i in tqdm(range(0,len(data),250000)):
#     if i+250000 <= len(data): 
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:i+250000]])
#     else:
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:]])
#     sql = "INSERT INTO gfdata.tl_sprd_rw_new ({col}) VALUES {i};".format(col = col,i=args_str)
#     start_time = time.time()
#     try:
#         cursor_gf.execute(sql)
#     except Exception as e:
#         print("Insert Error: ", e)
#     gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [32]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sprd_rw_new;
#                 ''')
# gf.commit()

## layer_group_4 건물통합GIS

In [33]:
# # layer_group_4_건물GIS
# # 전체 테이블과 지역별 테이블 같이 생성: 같은 반복문에 넣어야함
# # DB의 테이블명과 파일명이 불일치 -> 파일명과 테이블명을 매칭 시킬 수 있는 dic 필요
# # 한테이블에 여러 shp를 넣어야함 -> (1)(2)(3) 등의 파일은 하나의 테이블에 들어갈 수 있게 -> 지역명 이용합시다^^
# # 현재 51,52 테이블 없음-> 51(강원) : 42, 52(전북) : 45
# from tqdm import tqdm
# import time
# from shapely.geometry import MultiPolygon
# # 통 테이블 가져오기

# def convert_float_to_int_and_handle_null(gdf, column_name):
#     if gdf[column_name].dtype == 'float64':
#         gdf[column_name] = gdf[column_name].fillna(-1)
#         gdf[column_name] = gdf[column_name].astype(int)
#         gdf[column_name] = np.where(gdf[column_name] == -1, None, gdf[column_name])

# # Connect to PostgreSQL database


# path = '../layer_group_4_con/' #나머지 path(시도별 폴더명은 각각 넣어주기)
# # 현재 51,52 테이블 없음-> 51(강원) : 42, 52(전북) : 45으로 적재 후 테이블명 수기 조정 -> 나중에 코드로 추가
# li_folder_names = [] #folder 명 가져오기
# for i in os.listdir(path):
#     if (i != '.DS_Store') & (~('.zip' in i)):
#         li_folder_names.append(i)
# li_table_names = []# table명 가져오기
# cursor_gf.execute('''select RELNAME AS TABLE_NAME FROM PG_STAT_USER_TABLES \
#                   where RELNAME like '%_d010';''')
# li_tables = cursor_gf.fetchall() #윗 줄의 execute문과 더불어 테이블명 가져오기
# for i in li_tables:
#     li_table_names.append(i[0])
# li_table_names.remove('al_d010') #테이블명에서 전체 테이블 제거(al_d010)
# li_table_names.append('al_11_d010_new') #얘 혼자만 테이블명에 이미 _new가 붙어있음
# #전국테이블 생성
# sql = dic['al_d010_new']
# cursor_gf.execute(sql)
# gf.commit()

# #컬럼가져오기
# li_col = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'al_d010'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()##윗 줄의 execute문과 더불어 컬럼명 리스트 가져오기
# for j in range(len(li_tab_col)):
#     li_col.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li_col:
#     col = ' ,'.join([col,i])
# col = col[2:]
# n = 0
# for nm in li_table_names: #지역별 테이블 생성
#     gdf = 0
#     gdf_1 = 0
#     j = nm.split('_')[1]
#     sql = f'''CREATE TABLE gfdata.{nm}_new (
# 	gid int4 DEFAULT nextval('gfdata.{nm}_new'::regclass) NOT NULL,
# 	a0 int4 NULL,
# 	a1 varchar(28) NULL,
# 	a2 varchar(19) NULL,
# 	a3 varchar(10) NULL,
# 	a4 varchar(254) NULL,
# 	a5 varchar(10) NULL,
# 	a6 varchar(1) NULL,
# 	a7 varchar(254) NULL,
# 	a8 varchar(5) NULL,
# 	a9 varchar(254) NULL,
# 	a10 varchar(2) NULL,
# 	a11 varchar(254) NULL,
# 	a12 float8 NULL,
# 	a13 varchar(10) NULL,
# 	a14 float8 NULL,
# 	a15 float8 NULL,       
# 	a16 float8 NULL,
# 	a17 float8 NULL,
# 	a18 float8 NULL,
# 	a19 varchar(28) NULL,
# 	a20 varchar(2) NULL,
# 	a21 varchar(17) NULL,
# 	a22 date NULL,
# 	a23 varchar(5) NULL,
# 	a24 varchar(254) NULL,
# 	a25 varchar(254) NULL,
# 	a26 int4 NULL,
# 	a27 int4 NULL,
# 	a28 varchar(10) NULL,
# 	geom public.geometry(multipolygon, 5186) NULL,
# 	CONSTRAINT {nm}_new_{time_1[-2:]}_pkey PRIMARY KEY (gid)
#     )
#     TABLESPACE gf
#     ;
#     CREATE INDEX {nm}_new_{time_1[-2:]}_geom_idx ON gfdata.{nm}_new USING gist (geom);'''
#     cursor_gf.execute(sql)
#     gf.commit()
#     # j는 table_name에서 오기에 강원(51), 전북이 없음(52) -> process단에서 잡자....
#     #데이터 가져오기
#     li = [x for x in li_folder_names if ''.join([j,'_']) in x] #j가 지역 코드로 들어가는 폴더명 호출
#     # 조건문을 if ''.join([j,'_']) <- 이렇게 한 이유: 그냥 j만 넣으면 폴더명 뒤 날짜 때문에 해당없는 폴더가 들어가기도 함
#     print(li, end='')
#     for k in li:
#         n_1 = 0 #-> 전체 테이블에 대한 gid를 넣기 위함(파일 단위로 추가될 때마다 파일길이만큼 늘려줌)
#         file_path = ''.join([path, k]) 
#         file = gpd.read_file(file_path, encoding = 'cp949')
#         gdf = gpd.GeoDataFrame(file, geometry = 'geometry', crs ='EPSG:5186').reset_index()
#         # Convert and handle null values for column A27 #건축문 연령대 구분코드 1~11의 값을 가짐
#         convert_float_to_int_and_handle_null(gdf, 'A27') 
#         # Convert and handle null values for column A26 #건축문 연령 자연수
#         convert_float_to_int_and_handle_null(gdf, 'A26')
#         gdf.columns = li_col
#         gdf.set_geometry("geom", inplace=True)
#         print('DATA 가공')
#         for i in tqdm(range(len(gdf))):
#             try:
#                 polygon = wkt.loads(str(gdf.loc[i,'geom'])) #geom을 wkt에 로드
#             except:
#                 pass
#             try:
#                 m = MultiPolygon([polygon]) #멀티폴리곤으로 바꿔줄 수 있는 경우 바꿔준다
#                 gdf.loc[i, 'geom'] = m
#             except:
#                 pass
#             gdf_1 = gdf  
#         gdf['gid'] = gdf['gid'] + n_1
#         gdf_1['gid'] = gdf_1['gid'] + n
#         n+=len(gdf)
#         n_1+=len(gdf)
#         gdf.to_postgis( #gdf -> 전국 테이블
#             name='al_d010_new',
#             con=con_engine,
#             schema='gfdata',
#             if_exists='append',
#             dtype={'geom': Geometry('MULTIPOLIGON', srid=5186)}
#             )

#         gdf_1.to_postgis(
#             name=f'{nm}_new',
#             con=con_engine,
#             schema='gfdata',
#             if_exists='append',
#             dtype={'geom': Geometry('MULTIPOLIGON', srid=5186)}
#             ) 


In [ ]:
# for i in li_table_names:
#     cursor_gf.execute(f'''drop TABLE gfdata.{i}_new;
#                 ''')
#     gf.commit()
# cursor_gf.execute(f'''drop TABLE gfdata.al_d010_new;
#             ''')
# gf.commit()



In [35]:
li_table_names = []# table명 가져오기
cursor_gf.execute('''select RELNAME AS TABLE_NAME FROM PG_STAT_USER_TABLES \
                  where RELNAME like '%_d010';''')
li_tables = cursor_gf.fetchall() #윗 줄의 execute문과 더불어 테이블명 가져오기
for i in li_tables:
    li_table_names.append(i[0])
li_table_names.remove('al_d010') #테이블명에서 전체 테이블 제거(al_d010)
li_table_names.append('al_11_d010_new') #얘 혼자만 테이블명에 이미 _new가 붙어있음
# li_table_names: 뒤에 _new가 부터있움

17


In [36]:
li_table_names

['al_36_d010',
 'al_30_d010',
 'al_48_d010',
 'al_46_d010',
 'al_47_d010',
 'al_44_d010',
 'al_43_d010',
 'al_41_d010',
 'al_31_d010',
 'al_29_d010',
 'al_28_d010',
 'al_27_d010',
 'al_50_d010',
 'al_26_d010',
 'al_42_d010',
 'al_45_d010',
 'al_11_d010_new']

In [37]:
# 기존 table 뒤에 old 붙여주기 -> 연속지적도 버전임, 7월 업로드 시 업데이트
for i in li_table_names:
    cursor_gf.execute(f'''alter TABLE gfdata.{i} rename to {i}_202404;''')
    gf.commit()
# 적재한 table, db 상 이름으로 바꿔주기
for i in li_table_names:
    new = '_'.join([i, 'new'])
    cursor_gf.execute(f'''alter TABLE gfdata.{new} rename to {i};''')
    gf.commit()
# 마스터 테이블명 변경
# cursor_gf.execute('''alter TABLE gfdata.lsmd_cont_ldreg_all rename to lsmd_cont_ldreg_all_old;''')
# cursor_gf.execute('''alter TABLE gfdata.lsmd_cont_ldreg_new rename to lsmd_cont_ldreg_all;''')
# gf.commit()

## layer_group_4 도로명

In [35]:
# # 도로명주소-건물도형
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# path = '../layer_group_4_road/TL_SGCO_RNADR_MST' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass
# data = []
# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5179', geometry='geometry').reset_index()
#     gdf = gdf.to_crs('EPSG:5186')
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌
#     data.extend(d)
# print('DATA 가공')
# for i in tqdm(range(len(data))):
#     polygon = data[i][-1]
#     try:
#         data[i][-1] = str(from_shape(MultiPolygon([polygon]), srid=5186))
#     except:
#         data[i][-1] = str(from_shape(polygon, srid=5186)) 

# #테이블 생성
# sql = dic['tl_sgco_rnadr_mst_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# #insert

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_sgco_rnadr_mst'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]
# #---------------------------------------------------------------------------------------------------------

# #insert
# for i in tqdm(range(0,len(data),250000)):
#     if i+250000 <= len(data): 
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:i+250000]])
#     else:
#         args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in data[i:]])
#     sql = "INSERT INTO gfdata.tl_sgco_rnadr_mst_new ({col}) VALUES {i};".format(col = col,i=args_str)
#     start_time = time.time()
#     try:
#         cursor_gf.execute(sql)
#     except Exception as e:
#         print("Insert Error: ", e)
#     gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [36]:
# cursor_gf.execute('''drop TABLE gfdata.tl_sgco_rnadr_mst_new;
#                 ''')
# gf.commit()

In [37]:
# # 도로명주소-연결선정보
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# # int 컬럼에 nan이 들어가 있음 -> 건통스크립트 바탕으로 수정 필요
# path = '../layer_group_4_road/TL_SPOT_CNTC' #multipolygon
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_spot_cntc'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_spot_cntc_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()

# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5179', geometry='geometry').reset_index()
#     gdf = gdf.to_crs('EPSG:5186')

#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌

#     print('DATA 가공')

#     for i in range(len(d)):
#         polygon = d[i][-1]
#         try:
#             d[i][-1] = str(from_shape(MultiLineString([polygon]), srid=5186))
#         except:
#             d[i][-1] = str(from_shape(polygon, srid=5186)) 

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_spot_cntc_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [38]:
# cursor_gf.execute('''drop TABLE gfdata.tl_spot_cntc_new;
#                 ''')
# gf.commit()


In [39]:
# # 도로명주소-출입구정보
# # 분할된 파일로 data를 만드는 경우------------------------------------------------------------------------------ 
# # issue
# # 1. int, float 필등에 nan 값을 넣을 수 없음: 해당 내역을 str로 형변환하여 넣어야한다는데 가능한가
# # 2. gid에 대한 pk설정에 따른 오류: gid로 사용되는 index열에 대한 숫자 신경써야함
# path = '../layer_group_4_road/TL_SPBD_ENTRC' #Point
# li_folder = os.listdir(path) 
# try:
#     li_folder.remove('.DS_Store')
# except:
#     pass

# #컬럼구해오기-> 여기는 geondba 권한으로 
# #---------------------------------------------------------------------------------------------------------
# li = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'tl_spbd_entrc'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')
# li_tab_col = cursor.fetchall()
# for j in range(len(li_tab_col)):
#     li.append(li_tab_col[j][1]) #컬럼명 수집
# col = ''
# for i in li:
#     col = ' ,'.join([col,i])
# col = col[2:]

# #테이블 생성
# sql = dic['tl_spbd_entrc_new'].replace('\n','').replace('\t','')
# cursor_gf.execute(sql)
# gf.commit()


# d = []
# n = 0 #index를 먹이기 위함
# for i in tqdm(li_folder):
#     n += len(d)
#     file = gpd.read_file('/'.join([path,i]), encoding = 'cp949').dropna(subset = ['BUL_MAN_NO','ENT_MAN_NO','EQB_MAN_SN'], how = 'any')
#     gdf = gpd.GeoDataFrame(file, crs = 'EPSG:5179', geometry='geometry').reset_index()
#     gdf = gdf.to_crs('EPSG:5186')
#     n+=len(d)
#     d = [list(gdf.loc[i]) for i in range(len(gdf))]
#     for i in range(len(d)):
#           d[i][0] = d[i][0] + n #index가 분할된 파일마다 0으로 초기화되기에 파일의 길이만큼 더해줌
#           d[i][-1] = str(from_shape(d[i][-1])) 
#     print('DATA 가공')

#     #insert
#     for i in range(0,len(d),250000):
#         if i+250000 <= len(d): 
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:i+250000]])
#         else:
#             args_str = ", ".join([cursor_gf.mogrify('(%s,%s,%s,%s,%s,%s,%s,%s)', row).decode('utf-8') for row in d[i:]])
#         sql = "INSERT INTO gfdata.tl_spbd_entrc_new ({col}) VALUES {i};".format(col = col,i=args_str)
#         start_time = time.time()
#         try:
#             cursor_gf.execute(sql)
#         except Exception as e:
#             print("Insert Error: ", e)
#         gf.commit()
#     print("%s: %.5f secs" % ("Taken Time", time.time() - start_time))

In [40]:
# cursor_gf.execute('''drop TABLE gfdata.tl_spbd_entrc_new;
#                 ''')
# gf.commit()

# 건물통합전국데이터

In [56]:
# # layer_group_4_건물GIS
# # 전체 테이블과 지역별 테이블 같이 생성: 같은 반복문에 넣어야함
# # DB의 테이블명과 파일명이 불일치 -> 파일명과 테이블명을 매칭 시킬 수 있는 dic 필요
# # 한테이블에 여러 shp를 넣어야함 -> (1)(2)(3) 등의 파일은 하나의 테이블에 들어갈 수 있게 -> 지역명 이용합시다^^
# # 현재 51,52 테이블 없음-> 51(강원) : 42, 52(전북) : 45
# from tqdm import tqdm
# import time
# from shapely.geometry import MultiPolygon
# import pandas as pd
# # 통 테이블 가져오기

# def convert_float_to_int_and_handle_null(gdf, column_name):
#     if gdf[column_name].dtype == 'float64':
#         gdf[column_name] = gdf[column_name].fillna(-1)
#         gdf[column_name] = gdf[column_name].astype(int)
#         gdf[column_name] = np.where(gdf[column_name] == -1, None, gdf[column_name])

# # Connect to PostgreSQL database
# engine = con_engine

# file_path = '/Users/kownkihoon/Desktop/업무_wavus/0.GeOnAI/sql_chain/trial_data/GIS_con_data.csv'

# #전국테이블 생성
# sql = f'''
# CREATE TABLE gfdata.gis_con_data (
# 	gid int4 DEFAULT nextval('gfdata.gis_con_data'::regclass) NOT NULL,
# 	원천도형ID int4 NULL,
# 	GIS건물통합식별번호 varchar(254) NULL,
# 	고유번호 varchar(19) NULL,
# 	법정동코드 varchar(254) NULL,
# 	법정동명 varchar(254) NULL,
# 	지번 varchar(10) NULL,
# 	특수지코드 varchar(1) NULL,
# 	특수지구분명 varchar(254) NULL,
# 	건축물용도코드 varchar(254) NULL,
# 	건축물용도명 varchar(254) NULL,
# 	건축물구조코드 varchar(254) NULL,
# 	건축물구조명 varchar(254) NULL,
# 	건축물면적 float8 NULL,
# 	사용승인일자 varchar(10) NULL,
# 	연면적 float8 NULL,
# 	대지면적 float8 NULL,
# 	높이 float8 NULL,
# 	건폐율 float8 NULL,
# 	용적율 float8 NULL,
# 	건축물ID varchar(28) NULL,
# 	위반건축물여부 varchar(2) NULL,
# 	참조체계연계키 varchar(17) NULL,
# 	데이터기준일자 date NULL,
# 	원천시도시군구코드 varchar(254) NULL,
# 	건물명 varchar(254) NULL,
# 	건물동명 varchar(254) NULL,
# 	지상층_수 int4 NULL,
# 	지하층_수 int4 NULL,
# 	데이터생성변경일자 varchar(10) NULL,
#     사용승인연도 int4 NULL,
#     건축물나이 int4 NULL,
# 	geometry public.geometry(multipolygon, 5186) NULL,
# 	CONSTRAINT gis_con_data_pkey_{str(date_1)[-2:]+time_1[-2:]} PRIMARY KEY (gid)
# )
# TABLESPACE gf
# ;
# CREATE INDEX gis_con_data_geom_idx_{str(date_1)[-2:]+time_1[-2:]} ON gfdata.gis_con_data USING gist (geometry);
# CREATE INDEX gis_con_data_geom_idx1_{str(date_1)[-2:]+time_1[-2:]} ON gfdata.gis_con_data USING gist (geometry);
# '''
# print('테이블 생성')
# cursor_gf.execute(sql)
# gf.commit()

# # 컬럼가져오기
# li_col = []
# cursor.execute('''SELECT TABLE_NAME as 테이블이름, column_name as 컬럼명
#                 FROM INFORMATION_SCHEMA.COLUMNS 
#                 WHERE TABLE_CATALOG = 'geondb'
#                 and table_schema = 'gfdata'
#                 and table_name = 'gis_con_data'
#                 ORDER BY 테이블이름, ORDINAL_POSITION;''')

# li_tab_col = cursor.fetchall()##윗 줄의 execute문과 더불어 컬럼명 리스트 가져오기
# for j in range(len(li_tab_col)):
#     li_col.append(li_tab_col[j][1]) #컬럼명 수집
# #    데이터 가져오기
# df = pd.read_csv(file_path)
# gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geom']), crs='EPSG:5186').drop(columns=['geom'])
# gdf['건축물구조코드'] = gdf['건축물구조코드'].str.split('.').str[0]
# gdf.columns = li_col
# # Convert and handle null values for column A27 #건축문 연령대 구분코드 1~11의 값을 가짐
# convert_float_to_int_and_handle_null(gdf, '지상층_수') 
# # Convert and handle null values for column A26 #건축문 연령 자연수
# convert_float_to_int_and_handle_null(gdf, '지하층_수')

# gdf.set_geometry("geometry", inplace=True)
# print('DATA 가공')
# for i in tqdm(range(len(gdf))):
# 	try:
# 		polygon = wkt.loads(str(gdf.loc[i,'geometry'])) #geom을 wkt에 로드
# 	except:
# 		pass	
# 	try:
# 		m = MultiPolygon([polygon]) #멀티폴리곤으로 바꿔줄 수 있는 경우 바꿔준다
# 		gdf.loc[i, 'geometry'] = m
# 	except:
# 		pass


테이블 생성


In [57]:
# gdf.to_postgis( #gdf -> 전국 테이블
#     name='gis_con_data',
#     con=con_engine,
#     schema='gfdata',
#     if_exists='append',
#     dtype={'geometry': Geometry('MULTIPOLIGON', srid=5186)},
#     )

In [55]:
# cursor_gf.execute('''drop TABLE gfdata.gis_con_data;
#                 ''')
# gf.commit()